Creating a Retrieval-Augmented Generation (RAG) system using Azure Blob Storage involves several steps. Here, I'll provide a basic example to get you started. The code will cover uploading documents to Azure Blob Storage, indexing the documents using Azure Cognitive Search, and querying the indexed data using Azure OpenAI for generating responses.

### Prerequisites

1. **Azure Account**: You need an Azure account with access to Azure Blob Storage, Azure Cognitive Search, and Azure OpenAI.
2. **Azure SDK for Python**: Install the necessary Azure SDKs.
   ```bash
   pip install azure-storage-blob azure-search-documents openai
   ```

### Step 1: Upload Documents to Azure Blob Storage

First, upload your documents to Azure Blob Storage.

```python
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

# Replace with your Azure Blob Storage connection string and container name
connection_string = "your_connection_string"
container_name = "your_container_name"

blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

# Upload a document
def upload_document(file_path, blob_name):
    with open(file_path, "rb") as data:
        blob_client = container_client.get_blob_client(blob_name)
        blob_client.upload_blob(data)

upload_document("path/to/your/document.pdf", "document.pdf")
```

### Step 2: Index Documents with Azure Cognitive Search

Next, create an index and index the documents in Azure Cognitive Search.

```python
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex,
    SimpleField,
    SearchableField,
    IndexDocumentsBatch,
    IndexingResult,
    SearchIndexerDataSourceConnection,
    SearchIndexer,
    SearchIndexerSkillset,
    SearchIndexerSkill,
    SearchIndexerDataSourceType
)

# Replace with your Azure Cognitive Search details
search_service_endpoint = "your_search_service_endpoint"
search_api_key = "your_search_api_key"

index_client = SearchIndexClient(endpoint=search_service_endpoint, credential=search_api_key)
search_client = SearchClient(endpoint=search_service_endpoint, index_name="your_index_name", credential=search_api_key)

# Create an index
index_name = "your_index_name"
fields = [
    SimpleField(name="id", type="Edm.String", key=True),
    SearchableField(name="content", type="Edm.String")
]

index = SearchIndex(name=index_name, fields=fields)
index_client.create_index(index)

# Index documents
documents = [
    {"id": "1", "content": "This is a sample document content."},
    # Add more documents as needed
]

batch = IndexDocumentsBatch()
batch.add_upload_actions(documents)
results = search_client.index_documents(batch)
```

### Step 3: Query Indexed Data and Generate Responses

Finally, query the indexed data and generate responses using Azure OpenAI.

```python
import openai

# Replace with your OpenAI API key
openai.api_key = "your_openai_api_key"

def search_documents(query):
    results = search_client.search(query)
    return [result["content"] for result in results]

def generate_response(query):
    documents = search_documents(query)
    context = "\n".join(documents)
    prompt = f"Based on the following documents, answer the question: {query}\n\nDocuments:\n{context}\n\nAnswer:"

    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=150
    )

    return response.choices[0].text.strip()

query = "What is the content of the sample document?"
response = generate_response(query)
print(response)
```

### Summary

This code provides a basic structure for setting up a RAG system using Azure Blob Storage, Azure Cognitive Search, and Azure OpenAI. Adjust the details according to your specific needs and Azure configurations.